In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 8)

import collections

import warnings
# warnings.filterwarnings('ignore')
from matplotlib.backends.backend_pdf import PdfPages

from pathlib import Path

import db_queries as db
import vivarium_helpers.id_helper as idh

# Add the repo directory vivarium_research_ciff_sam/ to sys.path
import os, sys
repo_path = os.path.abspath('../..')
sys.path.append(repo_path)
# Assumes vivarium_research_ciff_sam/ is in sys.path
import model_validation.vivarium_transformed_output as vto
# import model_validation.vivarium_raw_output as vro
import model_validation.vivarium_output_processing as vop
import model_validation.ciff_sam_results as csr

!pwd
!whoami
!date

/ihme/homes/ndbs/vivarium_research_ciff_sam/nathaniel/scratch__gitignore__
ndbs
Mon Aug 30 13:12:45 PDT 2021


In [9]:
%load_ext autoreload
%autoreload 2

# Test my original code

In [3]:
orig_data = csr.VivariumMeasures.from_model_spec(2.4)
orig_data.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count']

In [13]:
data23 = csr.VivariumMeasures.from_model_spec(2.3)
data23.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count']

# Test addition of model 2.3

In [12]:
csr.models

,model_id,model_name,run_id
0,2.3,v2.3_wasting_birth_prevalence,2021_07_26_17_14_31
1,2.4,v2.4_corrected_fertility,2021_08_03_15_08_32
2,2.5,v2.5_stunting,2021_08_05_16_17_12
3,3.0,v3.0_sq_lns,2021_08_16_17_54_19
4,3.1,v3.1_sq_lns_stunting_stratified,2021_08_24_10_28_32


In [14]:
cdata23 = csr.VivariumMeasures.cleaned_from_model_spec(2.3)
cdata23.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count']

In [15]:
cdata23.wasting_state_person_time

,sex,year,wasting_state,measure,input_draw,scenario,value,age
0,female,2022,mild_child_wasting,state_person_time,29,baseline,67.629021,early_neonatal
1,female,2022,mild_child_wasting,state_person_time,29,baseline,242.324435,late_neonatal
2,female,2022,mild_child_wasting,state_person_time,29,baseline,3526.287474,1-5_months
3,female,2022,mild_child_wasting,state_person_time,29,baseline,8192.839151,6-11_months
...,...,...,...,...,...,...,...,...
2876,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,4633.670089,1-5_months
2877,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,5516.294319,6-11_months
2878,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,11032.581793,12_to_23_months
2879,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,63806.663929,2_to_4


In [16]:
cdata23.wasting_transition_count

,sex,year,measure,input_draw,scenario,value,age,transition
0,female,2022,transition_count,29,baseline,218.0,early_neonatal,mild_child_wasting_to_moderate_acute_malnutrition
1,female,2022,transition_count,29,baseline,677.0,late_neonatal,mild_child_wasting_to_moderate_acute_malnutrition
2,female,2022,transition_count,29,baseline,11499.0,1-5_months,mild_child_wasting_to_moderate_acute_malnutrition
3,female,2022,transition_count,29,baseline,26814.0,6-11_months,mild_child_wasting_to_moderate_acute_malnutrition
...,...,...,...,...,...,...,...,...
5036,male,2026,transition_count,946,baseline,479.0,1-5_months,susceptible_to_child_wasting_to_mild_child_was...
5037,male,2026,transition_count,946,baseline,674.0,6-11_months,susceptible_to_child_wasting_to_mild_child_was...
5038,male,2026,transition_count,946,baseline,1270.0,12_to_23_months,susceptible_to_child_wasting_to_mild_child_was...
5039,male,2026,transition_count,946,baseline,6814.0,2_to_4,susceptible_to_child_wasting_to_mild_child_was...


# Test my `get_all_causes_measure` function

In [17]:
data23.deaths

,sex,year,cause,measure,input_draw,scenario,value,wasting_state,age
0,female,2022,diarrheal_diseases,death,29,baseline,1.0,susceptible_to_child_wasting,early_neonatal
1,female,2022,diarrheal_diseases,death,29,baseline,0.0,mild_child_wasting,early_neonatal
2,female,2022,diarrheal_diseases,death,29,baseline,0.0,moderate_acute_malnutrition,early_neonatal
3,female,2022,diarrheal_diseases,death,29,baseline,0.0,severe_acute_malnutrition,early_neonatal
...,...,...,...,...,...,...,...,...,...
17276,male,2026,severe_acute_malnutrition,death,946,baseline,0.0,susceptible_to_child_wasting,2_to_4
17277,male,2026,severe_acute_malnutrition,death,946,baseline,0.0,mild_child_wasting,2_to_4
17278,male,2026,severe_acute_malnutrition,death,946,baseline,0.0,moderate_acute_malnutrition,2_to_4
17279,male,2026,severe_acute_malnutrition,death,946,baseline,0.0,severe_acute_malnutrition,2_to_4


In [18]:
csr.get_all_causes_measure(data23.deaths)

,sex,year,cause,measure,input_draw,scenario,value,wasting_state,age
0,female,2022,all_causes,death,29,baseline,22.0,mild_child_wasting,1-5_months
1,female,2023,all_causes,death,29,baseline,10.0,mild_child_wasting,1-5_months
2,female,2024,all_causes,death,29,baseline,10.0,mild_child_wasting,1-5_months
3,female,2025,all_causes,death,29,baseline,14.0,mild_child_wasting,1-5_months
...,...,...,...,...,...,...,...,...,...
2876,male,2023,all_causes,death,946,baseline,50.0,susceptible_to_child_wasting,late_neonatal
2877,male,2024,all_causes,death,946,baseline,38.0,susceptible_to_child_wasting,late_neonatal
2878,male,2025,all_causes,death,946,baseline,35.0,susceptible_to_child_wasting,late_neonatal
2879,male,2026,all_causes,death,946,baseline,51.0,susceptible_to_child_wasting,late_neonatal


# Test my `append_all_causes_burden` function

In [68]:
data25 = csr.VivariumMeasures.cleaned_from_model_spec(2.5)
data25.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count']

In [72]:
data25.append_all_causes_burden()
data25.deaths

,sex,year,cause,measure,input_draw,scenario,value,stunting_state,age
0,female,2022,diarrheal_diseases,death,29,baseline,32.0,cat4,early_neonatal
1,female,2022,diarrheal_diseases,death,29,baseline,0.0,cat3,early_neonatal
2,female,2022,diarrheal_diseases,death,29,baseline,0.0,cat2,early_neonatal
3,female,2022,diarrheal_diseases,death,29,baseline,0.0,cat1,early_neonatal
...,...,...,...,...,...,...,...,...,...
20156,male,2023,all_causes,death,946,baseline,739.0,cat4,late_neonatal
20157,male,2024,all_causes,death,946,baseline,662.0,cat4,late_neonatal
20158,male,2025,all_causes,death,946,baseline,667.0,cat4,late_neonatal
20159,male,2026,all_causes,death,946,baseline,683.0,cat4,late_neonatal


In [73]:
data25.ylds

,sex,year,cause,measure,input_draw,scenario,value,stunting_state,age
0,female,2022,diarrheal_diseases,ylds,29,baseline,6.012410,cat4,early_neonatal
1,female,2022,diarrheal_diseases,ylds,29,baseline,0.000000,cat3,early_neonatal
2,female,2022,diarrheal_diseases,ylds,29,baseline,0.000000,cat2,early_neonatal
3,female,2022,diarrheal_diseases,ylds,29,baseline,0.000000,cat1,early_neonatal
...,...,...,...,...,...,...,...,...,...
17276,male,2023,all_causes,ylds,946,baseline,48.165973,cat4,late_neonatal
17277,male,2024,all_causes,ylds,946,baseline,47.816215,cat4,late_neonatal
17278,male,2025,all_causes,ylds,946,baseline,48.532792,cat4,late_neonatal
17279,male,2026,all_causes,ylds,946,baseline,48.129904,cat4,late_neonatal


In [74]:
'all_causes' in data25.deaths['cause']

False

In [75]:
data25.deaths['cause'].unique()

array(['diarrheal_diseases', 'lower_respiratory_infections', 'measles',
       'moderate_acute_malnutrition', 'other_causes',
       'severe_acute_malnutrition', 'all_causes'], dtype=object)

In [76]:
# Ugh, list comprehensions work, but just doing `x in data25.deaths['cause']`
# checks if x is in thee index, not the values.
[x for x in data25.deaths['cause']][-10:]

['all_causes',
 'all_causes',
 'all_causes',
 'all_causes',
 'all_causes',
 'all_causes',
 'all_causes',
 'all_causes',
 'all_causes',
 'all_causes']

In [77]:
'all_causes' in data25.deaths['cause'] 

False

In [78]:
'measles' in data25.deaths['cause'] 

False

In [79]:
0 in data25.deaths['cause'] 

True

# Figure out why YLDs table has fewer rows than YLLs and deaths

## It's because it's missing `other_causes`

In [28]:
data25.ylds.cause.unique()

array(['diarrheal_diseases', 'lower_respiratory_infections', 'measles',
       'moderate_acute_malnutrition', 'severe_acute_malnutrition',
       'all_causes'], dtype=object)

In [29]:
data25.ylds.query("cause in ['moderate_acute_malnutrition', 'severe_acute_malnutrition']").age.unique()

array(['early_neonatal', 'late_neonatal', '1-5_months', '6-11_months',
       '12_to_23_months', '2_to_4'], dtype=object)

In [30]:
data25.ylls.query("cause in ['moderate_acute_malnutrition', 'severe_acute_malnutrition']").age.unique()

array(['early_neonatal', 'late_neonatal', '1-5_months', '6-11_months',
       '12_to_23_months', '2_to_4'], dtype=object)

In [31]:
data25.ylls.query("cause=='measles'").age.unique()

array(['early_neonatal', 'late_neonatal', '1-5_months', '6-11_months',
       '12_to_23_months', '2_to_4'], dtype=object)

In [32]:
data25.ylds.query("cause=='measles'").age.unique()

array(['early_neonatal', 'late_neonatal', '1-5_months', '6-11_months',
       '12_to_23_months', '2_to_4'], dtype=object)

In [35]:
len(data25.ylls) - len(data25.ylls.query("cause=='other_causes'"))

17280

# Compute SAM duration and inverse duration

In [36]:
data25.wasting_state_person_time

,sex,year,wasting_state,measure,input_draw,scenario,value,age
0,female,2022,mild_child_wasting,state_person_time,29,baseline,414.090349,early_neonatal
1,female,2022,mild_child_wasting,state_person_time,29,baseline,1234.535250,late_neonatal
2,female,2022,mild_child_wasting,state_person_time,29,baseline,8926.954141,1-5_months
3,female,2022,mild_child_wasting,state_person_time,29,baseline,10434.387406,6-11_months
...,...,...,...,...,...,...,...,...
2876,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,30258.041068,1-5_months
2877,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,35695.241615,6-11_months
2878,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,70879.949350,12_to_23_months
2879,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,215444.187543,2_to_4


In [37]:
data25.wasting_transition_count

,sex,year,measure,input_draw,scenario,value,age,transition
0,female,2022,transition_count,29,baseline,1245.0,early_neonatal,mild_child_wasting_to_moderate_acute_malnutrition
1,female,2022,transition_count,29,baseline,3803.0,late_neonatal,mild_child_wasting_to_moderate_acute_malnutrition
2,female,2022,transition_count,29,baseline,29178.0,1-5_months,mild_child_wasting_to_moderate_acute_malnutrition
3,female,2022,transition_count,29,baseline,34038.0,6-11_months,mild_child_wasting_to_moderate_acute_malnutrition
...,...,...,...,...,...,...,...,...
5036,male,2026,transition_count,946,baseline,3330.0,1-5_months,susceptible_to_child_wasting_to_mild_child_was...
5037,male,2026,transition_count,946,baseline,4391.0,6-11_months,susceptible_to_child_wasting_to_mild_child_was...
5038,male,2026,transition_count,946,baseline,8691.0,12_to_23_months,susceptible_to_child_wasting_to_mild_child_was...
5039,male,2026,transition_count,946,baseline,23097.0,2_to_4,susceptible_to_child_wasting_to_mild_child_was...


In [84]:
wasting_state_person_time_per_transition = vop.ratio(
    data25.wasting_state_person_time,
    data25.wasting_transition_count,
    strata=['year', 'sex', 'age'],
    numerator_broadcast=['wasting_state'],
    denominator_broadcast=['transition'],
)
wasting_state_person_time_per_transition

,year,sex,age,input_draw,scenario,wasting_state,transition,value,numerator_measure,denominator_measure,multiplier
0,2022,female,1-5_months,29,baseline,mild_child_wasting,mild_child_wasting_to_moderate_acute_malnutrition,0.305948,state_person_time,transition_count,1
1,2022,female,1-5_months,29,baseline,mild_child_wasting,mild_child_wasting_to_susceptible_to_child_was...,2.723293,state_person_time,transition_count,1
2,2022,female,1-5_months,29,baseline,mild_child_wasting,moderate_acute_malnutrition_to_mild_child_wasting,0.349830,state_person_time,transition_count,1
3,2022,female,1-5_months,29,baseline,mild_child_wasting,moderate_acute_malnutrition_to_severe_acute_ma...,1.390491,state_person_time,transition_count,1
...,...,...,...,...,...,...,...,...,...,...,...
20156,2026,male,late_neonatal,946,baseline,susceptible_to_child_wasting,moderate_acute_malnutrition_to_severe_acute_ma...,2.855497,state_person_time,transition_count,1
20157,2026,male,late_neonatal,946,baseline,susceptible_to_child_wasting,severe_acute_malnutrition_to_mild_child_wasting,5.162682,state_person_time,transition_count,1
20158,2026,male,late_neonatal,946,baseline,susceptible_to_child_wasting,severe_acute_malnutrition_to_moderate_acute_ma...,6.018126,state_person_time,transition_count,1
20159,2026,male,late_neonatal,946,baseline,susceptible_to_child_wasting,susceptible_to_child_wasting_to_mild_child_was...,10.455735,state_person_time,transition_count,1


In [40]:
data25.wasting_transition_count.transition.unique()

array(['mild_child_wasting_to_moderate_acute_malnutrition',
       'mild_child_wasting_to_susceptible_to_child_wasting',
       'moderate_acute_malnutrition_to_mild_child_wasting',
       'moderate_acute_malnutrition_to_severe_acute_malnutrition',
       'severe_acute_malnutrition_to_mild_child_wasting',
       'severe_acute_malnutrition_to_moderate_acute_malnutrition',
       'susceptible_to_child_wasting_to_mild_child_wasting'], dtype=object)

In [80]:
'moderate_acute_malnutrition_to_mild_child_wasting'.split('_to_')

['moderate_acute_malnutrition', 'mild_child_wasting']

In [81]:
'mild_child_wasting_to_susceptible_to_child_wasting'.split('_to_')

['mild_child_wasting', 'susceptible', 'child_wasting']

In [54]:
into_mam = [
    'mild_child_wasting_to_moderate_acute_malnutrition',
    'severe_acute_malnutrition_to_moderate_acute_malnutrition'
]
mam_duration = wasting_state_person_time_per_transition.query(
    "wasting_state=='moderate_acute_malnutrition' and "
    "transition in @into_mam"
)
mam_duration # WRONG!! 

,year,sex,age,input_draw,scenario,wasting_state,transition,value,numerator_measure,denominator_measure,multiplier
7,2022,female,1-5_months,29,baseline,moderate_acute_malnutrition,mild_child_wasting_to_moderate_acute_malnutrition,0.121150,state_person_time,transition_count,1
12,2022,female,1-5_months,29,baseline,moderate_acute_malnutrition,severe_acute_malnutrition_to_moderate_acute_ma...,1.226973,state_person_time,transition_count,1
35,2022,female,1-5_months,223,baseline,moderate_acute_malnutrition,mild_child_wasting_to_moderate_acute_malnutrition,0.117157,state_person_time,transition_count,1
40,2022,female,1-5_months,223,baseline,moderate_acute_malnutrition,severe_acute_malnutrition_to_moderate_acute_ma...,1.072641,state_person_time,transition_count,1
...,...,...,...,...,...,...,...,...,...,...,...
20111,2026,male,late_neonatal,829,baseline,moderate_acute_malnutrition,mild_child_wasting_to_moderate_acute_malnutrition,0.113515,state_person_time,transition_count,1
20116,2026,male,late_neonatal,829,baseline,moderate_acute_malnutrition,severe_acute_malnutrition_to_moderate_acute_ma...,0.828246,state_person_time,transition_count,1
20139,2026,male,late_neonatal,946,baseline,moderate_acute_malnutrition,mild_child_wasting_to_moderate_acute_malnutrition,0.112981,state_person_time,transition_count,1
20144,2026,male,late_neonatal,946,baseline,moderate_acute_malnutrition,severe_acute_malnutrition_to_moderate_acute_ma...,0.854586,state_person_time,transition_count,1


In [43]:
sam_duration = wasting_state_person_time_per_transition.query(
    "wasting_state=='severe_acute_malnutrition' and "
    "transition=='moderate_acute_malnutrition_to_severe_acute_malnutrition'"
)
sam_duration

,year,sex,age,input_draw,scenario,wasting_state,transition,value,numerator_measure,denominator_measure,multiplier
17,2022,female,1-5_months,29,baseline,severe_acute_malnutrition,moderate_acute_malnutrition_to_severe_acute_ma...,0.143876,state_person_time,transition_count,1
45,2022,female,1-5_months,223,baseline,severe_acute_malnutrition,moderate_acute_malnutrition_to_severe_acute_ma...,0.146593,state_person_time,transition_count,1
73,2022,female,1-5_months,232,baseline,severe_acute_malnutrition,moderate_acute_malnutrition_to_severe_acute_ma...,0.146573,state_person_time,transition_count,1
101,2022,female,1-5_months,357,baseline,severe_acute_malnutrition,moderate_acute_malnutrition_to_severe_acute_ma...,0.135437,state_person_time,transition_count,1
...,...,...,...,...,...,...,...,...,...,...,...
20065,2026,male,late_neonatal,650,baseline,severe_acute_malnutrition,moderate_acute_malnutrition_to_severe_acute_ma...,0.172432,state_person_time,transition_count,1
20093,2026,male,late_neonatal,680,baseline,severe_acute_malnutrition,moderate_acute_malnutrition_to_severe_acute_ma...,0.150054,state_person_time,transition_count,1
20121,2026,male,late_neonatal,829,baseline,severe_acute_malnutrition,moderate_acute_malnutrition_to_severe_acute_ma...,0.159562,state_person_time,transition_count,1
20149,2026,male,late_neonatal,946,baseline,severe_acute_malnutrition,moderate_acute_malnutrition_to_severe_acute_ma...,0.151926,state_person_time,transition_count,1


In [49]:
1/sam_duration.set_index(sam_duration.columns.difference(['value']).to_list())

value
age           denominator_measure input_draw multiplier numerator_measure scenario sex    transition                                         wasting_state             year          
1-5_months    transition_count    29         1          state_person_time baseline female moderate_acute_malnutrition_to_severe_acute_mal... severe_acute_malnutrition 2022  6.950410
                                  223        1          state_person_time baseline female moderate_acute_malnutrition_to_severe_acute_mal... severe_acute_malnutrition 2022  6.821591
                                  232        1          state_person_time baseline female moderate_acute_malnutrition_to_severe_acute_mal... severe_acute_malnutrition 2022  6.822543
                                  357        1          state_person_time baseline female moderate_acute_malnutrition_to_severe_acute_mal... severe_acute_malnutrition 2022  7.383500
...                                                                                                                                                                               ...
late_neonatal transition_count    650        1          state_person_time baseline male   moderate_acute_malnutrition_to_severe_acute_mal... severe_acute_malnutrition 2026  5.799380
                                  680        1          state_person_time baseline male   moderate_acute_malnutrition_to_severe_acute_mal... severe_acute_malnutrition 2026  6.664285
                                  829        1          state_person_time baseline male   moderate_acute_malnutrition_to_severe_acute_mal... severe_acute_malnutrition 2026  6.267150
                                  946        1          state_person_time baseline male   moderate_acute_malnutrition_to_severe_acute_mal... severe_acute_malnutrition 2026  6.582144

[720 rows x 1 columns]

# Write a `value` function to be able to easily do math on dataframes

In [124]:
def value1(df, include=None, exclude=None, value_cols=vop.VALUE_COLUMN):
    """Set index to all columns except value, sort of.
    The idea is to allow only one of include or exclude, but not both, but that is not enforced here.
    Also, include and exclude need to be lists not singletons.
    """
    value_cols = vop._ensure_iterable(value_cols, df)
    if exclude is None:
        exclude=[]
    if include is None:
        index_cols = df.columns.difference([*value_cols, *exclude]).to_list()
    else:
        index_cols = [*include, *vop.INDEX_COLUMNS]
    return df.set_index(index_cols)[value_cols]

def value2(df, include=None, exclude=None, value_cols=vop.VALUE_COLUMN):
    """Attempt to allow both include and exclude -- not sure if this makes sense..."""
    if include is not None:
        pass # didn't finish writing this part...
    include = vop._ensure_iterable(include, df)
    exclude = vop._ensure_iterable(exclude, df)
    value_cols = vop._ensure_iterable(value_cols, df)
    df = vop._ensure_columns_not_levels(df, [*include, *exclude])
    index_cols = df.columns.difference([*value_cols, *exclude]).to_list()
    index_cols += [*include, *vop.INDEX_COLUMNS]
    return df.set_index(index_cols)[value_cols]

def value(df, include=None, exclude=None, value_cols=vop.VALUE_COLUMN):
    """Set the index of the dataframe so that its only column(s) is (are) value_cols.
    This is useful for performing arithmetic on the dataframe, e.g. value(df1) + value(df2),
    assuming the resulting dataframes have compatible indices.
    - If neither `include` nor `exclude` are specified, the index of the dataframe will be
      set to all columns except those in `value_cols`.
    - If `include` is not None, the index will be set to the columns specified in `include`
      plus those specified in INDEX_COLUMNS (typically DRAW_COLUMN and SCENARIO_COLUMN).
    - If `exclude` is not None, the columns listed in `exclude` will be excluded from the index.
    - If both `include` and `exclude` are specified, a ValueError is raised - you can specify either
      columns to include or exclude, but not both.
    """
    value_cols = vop._ensure_iterable(value_cols, df)
    if include is None:
        exclude = vop._ensure_iterable(exclude, df, default=[])
        index_cols = df.columns.difference([*value_cols, *exclude]).to_list()
    elif exclude is not None:
        raise ValueError(
            "Only one of `include` or `exclude` can be specified."
            f" You passed {include=}, {exclude=}")
    else:
        include = vop._ensure_iterable(include, df)
        index_cols = [*include, *vop.INDEX_COLUMNS]
#     print(index_cols)
    return df.set_index(index_cols)[value_cols]

value(sam_duration)

value
age           denominator_measure input_draw multiplier numerator_measure scenario sex    transition                                         wasting_state             year          
1-5_months    transition_count    29         1          state_person_time baseline female moderate_acute_malnutrition_to_severe_acute_mal... severe_acute_malnutrition 2022  0.143876
                                  223        1          state_person_time baseline female moderate_acute_malnutrition_to_severe_acute_mal... severe_acute_malnutrition 2022  0.146593
                                  232        1          state_person_time baseline female moderate_acute_malnutrition_to_severe_acute_mal... severe_acute_malnutrition 2022  0.146573
                                  357        1          state_person_time baseline female moderate_acute_malnutrition_to_severe_acute_mal... severe_acute_malnutrition 2022  0.135437
...                                                                                                                                                                               ...
late_neonatal transition_count    650        1          state_person_time baseline male   moderate_acute_malnutrition_to_severe_acute_mal... severe_acute_malnutrition 2026  0.172432
                                  680        1          state_person_time baseline male   moderate_acute_malnutrition_to_severe_acute_mal... severe_acute_malnutrition 2026  0.150054
                                  829        1          state_person_time baseline male   moderate_acute_malnutrition_to_severe_acute_mal... severe_acute_malnutrition 2026  0.159562
                                  946        1          state_person_time baseline male   moderate_acute_malnutrition_to_severe_acute_mal... severe_acute_malnutrition 2026  0.151926

[720 rows x 1 columns]

In [56]:
vop.describe(1/value(sam_duration))

count  \
age           denominator_measure multiplier numerator_measure scenario sex    transition                                         wasting_state             year          
1-5_months    transition_count    1          state_person_time baseline female moderate_acute_malnutrition_to_severe_acute_mal... severe_acute_malnutrition 2022   12.0   
                                                                                                                                                            2023   12.0   
                                                                                                                                                            2024   12.0   
                                                                                                                                                            2025   12.0   
...                                                                                                                                                                 ...   
late_neonatal transition_count    1          state_person_time baseline male   moderate_acute_malnutrition_to_severe_acute_mal... severe_acute_malnutrition 2023   12.0   
                                                                                                                                                            2024   12.0   
                                                                                                                                                            2025   12.0   
                                                                                                                                                            2026   12.0   

                                                                                                                                                                      mean  \
age           denominator_measure multiplier numerator_measure scenario sex    transition                                         wasting_state             year             
1-5_months    transition_count    1          state_person_time baseline female moderate_acute_malnutrition_to_severe_acute_mal... severe_acute_malnutrition 2022  6.759653   
                                                                                                                                                            2023  6.813434   
                                                                                                                                                            2024  6.803735   
                                                                                                                                                            2025  6.798936   
...                                                                                                                                                                    ...   
late_neonatal transition_count    1          state_person_time baseline male   moderate_acute_malnutrition_to_severe_acute_mal... severe_acute_malnutrition 2023  6.512297   
                                                                                                                                                            2024  6.584634   
                                                                                                                                                            2025  6.469740   
                                                                                                                                                            2026  6.571454   

                                                                                                                                                                       std  \
age           denominator_measure multiplier numerator_measure scenario sex    transition                                         wasting_state             year             
1-5_months    tra

# Try to compute MAM duration

In [87]:
data25.wasting_transition_count.transition.unique()

array(['mild_child_wasting_to_moderate_acute_malnutrition',
       'mild_child_wasting_to_susceptible_to_child_wasting',
       'moderate_acute_malnutrition_to_mild_child_wasting',
       'moderate_acute_malnutrition_to_severe_acute_malnutrition',
       'severe_acute_malnutrition_to_mild_child_wasting',
       'severe_acute_malnutrition_to_moderate_acute_malnutrition',
       'susceptible_to_child_wasting_to_mild_child_wasting'], dtype=object)

In [91]:
data25.wasting_transition_count.transition[
    data25.wasting_transition_count.transition.str.endswith('moderate_acute_malnutrition')
].unique()

array(['mild_child_wasting_to_moderate_acute_malnutrition',
       'severe_acute_malnutrition_to_moderate_acute_malnutrition'],
      dtype=object)

In [92]:
data25.wasting_transition_count.transition.str.split('_to_')

0       [mild_child_wasting, moderate_acute_malnutrition]
1       [mild_child_wasting, moderate_acute_malnutrition]
2       [mild_child_wasting, moderate_acute_malnutrition]
3       [mild_child_wasting, moderate_acute_malnutrition]
                              ...                        
5036     [susceptible, child_wasting, mild_child_wasting]
5037     [susceptible, child_wasting, mild_child_wasting]
5038     [susceptible, child_wasting, mild_child_wasting]
5039     [susceptible, child_wasting, mild_child_wasting]
Name: transition, Length: 5040, dtype: object

In [93]:
data25.wasting_transition_count

,sex,year,measure,input_draw,scenario,value,age,transition
0,female,2022,transition_count,29,baseline,1245.0,early_neonatal,mild_child_wasting_to_moderate_acute_malnutrition
1,female,2022,transition_count,29,baseline,3803.0,late_neonatal,mild_child_wasting_to_moderate_acute_malnutrition
2,female,2022,transition_count,29,baseline,29178.0,1-5_months,mild_child_wasting_to_moderate_acute_malnutrition
3,female,2022,transition_count,29,baseline,34038.0,6-11_months,mild_child_wasting_to_moderate_acute_malnutrition
...,...,...,...,...,...,...,...,...
5036,male,2026,transition_count,946,baseline,3330.0,1-5_months,susceptible_to_child_wasting_to_mild_child_was...
5037,male,2026,transition_count,946,baseline,4391.0,6-11_months,susceptible_to_child_wasting_to_mild_child_was...
5038,male,2026,transition_count,946,baseline,8691.0,12_to_23_months,susceptible_to_child_wasting_to_mild_child_was...
5039,male,2026,transition_count,946,baseline,23097.0,2_to_4,susceptible_to_child_wasting_to_mild_child_was...


In [94]:
into_mam = [
    'mild_child_wasting_to_moderate_acute_malnutrition',
    'severe_acute_malnutrition_to_moderate_acute_malnutrition'
]
data25.wasting_transition_count.query("transition in @into_mam")

,sex,year,measure,input_draw,scenario,value,age,transition
0,female,2022,transition_count,29,baseline,1245.0,early_neonatal,mild_child_wasting_to_moderate_acute_malnutrition
1,female,2022,transition_count,29,baseline,3803.0,late_neonatal,mild_child_wasting_to_moderate_acute_malnutrition
2,female,2022,transition_count,29,baseline,29178.0,1-5_months,mild_child_wasting_to_moderate_acute_malnutrition
3,female,2022,transition_count,29,baseline,34038.0,6-11_months,mild_child_wasting_to_moderate_acute_malnutrition
...,...,...,...,...,...,...,...,...
4964,male,2026,transition_count,946,baseline,5523.0,1-5_months,severe_acute_malnutrition_to_moderate_acute_ma...
4965,male,2026,transition_count,946,baseline,6243.0,6-11_months,severe_acute_malnutrition_to_moderate_acute_ma...
4966,male,2026,transition_count,946,baseline,9510.0,12_to_23_months,severe_acute_malnutrition_to_moderate_acute_ma...
4967,male,2026,transition_count,946,baseline,17241.0,2_to_4,severe_acute_malnutrition_to_moderate_acute_ma...


In [129]:
mild_to_mam = data25.wasting_transition_count.query(
    "transition == 'mild_child_wasting_to_moderate_acute_malnutrition'")
sam_to_mam = data25.wasting_transition_count.query(
    "transition == 'severe_acute_malnutrition_to_moderate_acute_malnutrition'")
print(mild_to_mam.shape, sam_to_mam.shape)

(720, 8) (720, 8)


In [130]:
value(mild_to_mam)

,,,,,,,value
age,input_draw,measure,scenario,sex,transition,year,
early_neonatal,29,transition_count,baseline,female,mild_child_wasting_to_moderate_acute_malnutrition,2022,1245.0
late_neonatal,29,transition_count,baseline,female,mild_child_wasting_to_moderate_acute_malnutrition,2022,3803.0
1-5_months,29,transition_count,baseline,female,mild_child_wasting_to_moderate_acute_malnutrition,2022,29178.0
6-11_months,29,transition_count,baseline,female,mild_child_wasting_to_moderate_acute_malnutrition,2022,34038.0
...,...,...,...,...,...,...,...
1-5_months,946,transition_count,baseline,male,mild_child_wasting_to_moderate_acute_malnutrition,2026,39043.0
6-11_months,946,transition_count,baseline,male,mild_child_wasting_to_moderate_acute_malnutrition,2026,44876.0
12_to_23_months,946,transition_count,baseline,male,mild_child_wasting_to_moderate_acute_malnutrition,2026,82446.0
2_to_4,946,transition_count,baseline,male,mild_child_wasting_to_moderate_acute_malnutrition,2026,203069.0


In [131]:
value(sam_to_mam)

,,,,,,,value
age,input_draw,measure,scenario,sex,transition,year,
early_neonatal,29,transition_count,baseline,female,severe_acute_malnutrition_to_moderate_acute_malnutrition,2022,142.0
late_neonatal,29,transition_count,baseline,female,severe_acute_malnutrition_to_moderate_acute_malnutrition,2022,371.0
1-5_months,29,transition_count,baseline,female,severe_acute_malnutrition_to_moderate_acute_malnutrition,2022,2881.0
6-11_months,29,transition_count,baseline,female,severe_acute_malnutrition_to_moderate_acute_malnutrition,2022,3366.0
...,...,...,...,...,...,...,...
1-5_months,946,transition_count,baseline,male,severe_acute_malnutrition_to_moderate_acute_malnutrition,2026,5523.0
6-11_months,946,transition_count,baseline,male,severe_acute_malnutrition_to_moderate_acute_malnutrition,2026,6243.0
12_to_23_months,946,transition_count,baseline,male,severe_acute_malnutrition_to_moderate_acute_malnutrition,2026,9510.0
2_to_4,946,transition_count,baseline,male,severe_acute_malnutrition_to_moderate_acute_malnutrition,2026,17241.0


In [132]:
transitions_into_mam = value(mild_to_mam, exclude='transition') + value(sam_to_mam, exclude='transition')
transitions_into_mam

,,,,,,value
age,input_draw,measure,scenario,sex,year,
early_neonatal,29,transition_count,baseline,female,2022,1387.0
late_neonatal,29,transition_count,baseline,female,2022,4174.0
1-5_months,29,transition_count,baseline,female,2022,32059.0
6-11_months,29,transition_count,baseline,female,2022,37404.0
...,...,...,...,...,...,...
1-5_months,946,transition_count,baseline,male,2026,44566.0
6-11_months,946,transition_count,baseline,male,2026,51119.0
12_to_23_months,946,transition_count,baseline,male,2026,91956.0
2_to_4,946,transition_count,baseline,male,2026,220310.0


In [135]:
mam_duration=vop.ratio(
    data25.wasting_state_person_time.query("wasting_state=='moderate_acute_malnutrition'"),
    transitions_into_mam,
    strata=['year', 'sex', 'age'],
#     multiplier=365
)
mam_duration

,year,sex,age,input_draw,scenario,value,numerator_measure,denominator_measure,multiplier
0,2022,female,1-5_months,29,baseline,0.110263,state_person_time,transition_count,1
1,2022,female,1-5_months,223,baseline,0.105621,state_person_time,transition_count,1
2,2022,female,1-5_months,232,baseline,0.109135,state_person_time,transition_count,1
3,2022,female,1-5_months,357,baseline,0.106226,state_person_time,transition_count,1
...,...,...,...,...,...,...,...,...,...
716,2026,male,late_neonatal,650,baseline,0.104895,state_person_time,transition_count,1
717,2026,male,late_neonatal,680,baseline,0.097768,state_person_time,transition_count,1
718,2026,male,late_neonatal,829,baseline,0.099833,state_person_time,transition_count,1
719,2026,male,late_neonatal,946,baseline,0.099789,state_person_time,transition_count,1


In [133]:
transitions_into_mam.equals(
    vop.value(mild_to_mam, exclude='transition') + vop.value(sam_to_mam, exclude='transition')
)

True

In [127]:
vop.value(mild_to_mam, include=['age', 'sex', 'year'])

,,,,,value
age,sex,year,input_draw,scenario,
early_neonatal,female,2022,29,baseline,1245.0
late_neonatal,female,2022,29,baseline,3803.0
1-5_months,female,2022,29,baseline,29178.0
6-11_months,female,2022,29,baseline,34038.0
...,...,...,...,...,...
1-5_months,male,2026,946,baseline,39043.0
6-11_months,male,2026,946,baseline,44876.0
12_to_23_months,male,2026,946,baseline,82446.0
2_to_4,male,2026,946,baseline,203069.0


In [126]:
vop.value(mild_to_mam, include=['age'], exclude='sex')

ValueError: Only one of `include` or `exclude` can be specified. You passed include=['age'], exclude='sex'

In [128]:
sum([value(mild_to_mam, exclude='transition'), value(sam_to_mam, exclude='transition')])

,,,,,,value
age,input_draw,measure,scenario,sex,year,
early_neonatal,29,transition_count,baseline,female,2022,1387.0
late_neonatal,29,transition_count,baseline,female,2022,4174.0
1-5_months,29,transition_count,baseline,female,2022,32059.0
6-11_months,29,transition_count,baseline,female,2022,37404.0
...,...,...,...,...,...,...
1-5_months,946,transition_count,baseline,male,2026,44566.0
6-11_months,946,transition_count,baseline,male,2026,51119.0
12_to_23_months,946,transition_count,baseline,male,2026,91956.0
2_to_4,946,transition_count,baseline,male,2026,220310.0
